# Домашнее задание 3. Классификация имен

## Работу выполнили Ковалев Евгений и Сухарев Иван, 4 группа

В данном домашнем задании мы разработали два классификатора (наивный байесовский с использованием n-грамм и сеть с двумя слоями LSTM), которые определяют пол по имени.

*1. [1 балл] Предварительная обработка данных: 1) удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими одновременно), если такие есть; 2) создайте тестовое множество по следующему принципу: 20% от общего количества имен на каждую букву (т.е. 20% от имен на букву A, 20% имен на букву B, и т.д.)*

In [39]:
import pandas as pd
import glob
import re
import random
from nltk.util import ngrams
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy
from keras.models import Sequential
import keras.metrics
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
import numpy as np
import sklearn.metrics

Извлекаем имена из соответствующих текстовых файлов.

In [6]:
f = ' '.join(''.join(open('hw3_data/female.txt').readlines()).split('\n')).split()
m = ' '.join(''.join(open('hw3_data/male.txt').readlines()).split('\n')).split()

Избавляемся от неоднозначных имен.

In [7]:
female = [x for x in f if x not in m]
male = [x for x in m if x not in f]

Создаем массив из пар имя-пол.

In [8]:
labeled_names = sorted([(name, 'female') for name in female] + [(name, 'male') for name in male])

Находим все заглавные буквы, которые встречаются в массиве имен.

In [9]:
capital_letters = set(re.findall('[A-Z]', ''.join(female + male)))

Разбиваем выборку на обучающую и тестовую.

In [10]:
train = []
test = []
for letter in capital_letters:
    temp_names = [x for x in labeled_names if x[0][0] == letter]
    random_sample_names = random.sample(temp_names, int(round(len(temp_names) / 5)))
    test.extend(random_sample_names)
    train.extend([x for x in temp_names if x not in random_sample_names])

Перемешиваем элементы в обучающей и тестовой выборках.

In [11]:
random.shuffle(train)
random.shuffle(test)

*2. [4 балла] Используйте метод наивного Байеса для классификации имен: в качестве признаков используйте символьные n-граммы. Сравните результаты, получаемые при разных n = 2, 3, 4 по F-мере и аккуратности. В каких случаях метод ошибается? Для генерации n-грамм используйте from nltk.util import ngrams.*

Напишем функцию, которая будет выдавать результаты классификации с помощью метода наивного Байеса и с использованием n-грамм.

In [102]:
def names_bayes_classification(n):
    # массив всех n-грамм для обучающей выборки
    n_grams = []
    for item in train:
        name_n_grams = list(ngrams(item[0], n, pad_left=True, pad_right=True,\
                                   left_pad_symbol='<s>', right_pad_symbol='</s>'))
        n_grams.extend(name_n_grams)
    n_grams_set = set(n_grams)
    # делаем из множества всех n-грамм словарь признаков, где ключами (признаками) являются сами
    # n-граммы, а значениями - True/False (пока везде False)
    n_grams_features = dict.fromkeys(n_grams_set, False)
    
    # функция, которая получает на вход имя и выводит признаки для него (ставит в словаре признаков
    # значения True к тем n-граммам, которые встречаются в данном имени)
    def name2features(name):
        name_features = dict.fromkeys(n_grams_set, False)
        name_n_grams = list(ngrams(name, n, pad_left=True, pad_right=True,\
                                   left_pad_symbol='<s>', right_pad_symbol='</s>'))
        for n_gram in name_n_grams:
            name_features[n_gram] = True
        return name_features
    
    # классификация
    train_set = [(name2features(name), gender) for (name, gender) in train]
    test_set = [(name2features(name), gender) for (name, gender) in test]
    classifier = NaiveBayesClassifier.train(train_set)
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    for item in test:
        classifier_gender = classifier.classify(name2features(item[0]))
        if item[1] == classifier_gender and item[1] == 'male':
            true_positive += 1
        elif item[1] == classifier_gender and item[1] == 'female':
            true_negative += 1
        elif item[1] != classifier_gender and item[1] == 'female':
            false_positive += 1
        elif item[1] != classifier_gender and item[1] == 'male':
            false_negative += 1
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    f_score = (2 * precision * recall) / (precision + recall)
    print('Accuracy: {}'.format(accuracy(classifier, test_set)))
    print('True Positive: {}'.format(true_positive))
    print('True Negative: {}'.format(true_negative))
    print('False Positive: {}'.format(false_positive))
    print('False Negative: {}'.format(false_negative))
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))
    print('F-score: {}'.format(f_score))

In [103]:
print('2-GRAMS')
names_bayes_classification(2)

2-GRAMS
Accuracy: 0.8473282442748091
True Positive: 376
True Negative: 845
False Positive: 111
False Negative: 109
Precision: 0.7720739219712526
Recall: 0.7752577319587629
F-score: 0.7736625514403292


In [106]:
print('3-GRAMS')
names_bayes_classification(3)

3-GRAMS
Accuracy: 0.897293546148508
True Positive: 408
True Negative: 885
False Positive: 71
False Negative: 77
Precision: 0.8517745302713987
Recall: 0.8412371134020619
F-score: 0.8464730290456431


In [108]:
print('4-GRAMS')
names_bayes_classification(4)

4-GRAMS
Accuracy: 0.8979875086745316
True Positive: 404
True Negative: 890
False Positive: 66
False Negative: 81
Precision: 0.8595744680851064
Recall: 0.8329896907216495
F-score: 0.8460732984293194


Как видно, модель, использующая биграммы, проигрывает в качестве моделям, использующим 3-граммы и 4-граммы, которые в очередь показывают почти один и тот же результат.

*3.	[4 балла] Используйте сеть с двумя слоями LSTM для определения пола. Представление имени для классификации в этом случае: 2-мерный бинарный вектор количество букв в алфавите максимальная длина имени. Обозначим его через x. Если первая буква имени a, то x[1][1] = 1, если вторая – b, то x[2][1] = 1. Не забудьте про регуляризацию нейронной сети дропаутами. Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor/blob/master/train_genders.py. Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по F -мере и аккуратности. В каких случаях нейронная сеть ошибается?*

In [12]:
max_male = max([len(c) for c in male])
max_female = max([len(c) for c in female])
print('max_male: ' + str(max_male))
print('max_female: ' + str(max_female))

max_male: 15
max_female: 15


In [13]:
def transf(word):
    w = word.lower()
    ans = [[0 for c in range(max_male)] for i in range(26)]
    for i in range(len(w)):
        if ord(w[i]) - 97 >= 0:
            ans[ord(w[i]) - 97][i] = 1
    return ans

def conv(arr):
    x = [transf(x[0]) for x in arr]
    y = [[0, 1] if x[1] == 'female' else [1, 0] for x in arr]
    return np.array(x, dtype=np.float32), np.array(y, dtype=np.float32)

In [14]:
X_train, Y_train = conv(train)
X_test, Y_test = conv(test)

In [23]:
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(26, max_male)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy', 'fbeta_score'])


model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, verbose = 0)

print("Done!")
score = model.evaluate(X_test, Y_test, batch_size=32, verbose = 0)
print("Score " , score)

Build model...
Done!
Score  [0.47387077806725592, 0.77099236641221369, 0.7709923067248422]


In [30]:
def nconv(arr):
    return [round(x[0]) for x in arr]

F мера

In [40]:
sklearn.metrics.f1_score(nconv(model.predict(X_test)), nconv(Y_test))

0.67836257309941528

Accuracy

In [41]:
sklearn.metrics.accuracy_score(nconv(model.predict(X_test)), nconv(Y_test))

0.77099236641221369

*4.	[1 балл] Сравните результаты классификации разными методами. Какой метод лучше и почему?*

В данном случае точно можно выделить, что классификация с n-граммами показала куда лучшие результаты. Отметим, что в силу вычислительных сложностей не имели возможности тщательно обработать случай с нейронными сетями, но разница слишком значительна.